In [1]:
import os, math, serial, time, emerge_classes, csv
from dynamixel_sdk import *                    # Uses Dynamixel SDK library
from joint_handler import *
from zmqRemoteApi import RemoteAPIClient

from random import randint as ri
from random import uniform as ru

first = True

In [2]:
isSim = False
if isSim:
    cluster = False
    setup = Sim_setup(cluster)
    client = RemoteAPIClient('localhost', setup.port_conection)
    sim = client.getObject('sim')
    jointHandler = JointHandler(isSim, client, sim)
else:
    setup = EMERGE_setup()
    portHandler = PortHandler(setup.DEVICENAME)
    packetHandler = PacketHandler(setup.PROTOCOL_VERSION)
    jointHandler = JointHandler(isSim, portHandler, packetHandler)

In [27]:
class SensorHandler:
    def __init__(self, arduino_baud = 250000, arduino_port = 'COM6'):
        self.port = arduino_port
        self.baudrate = arduino_baud
        self.arduino = 0
        self.IR_command = 'IR'
        self.IMUg_command = 'Gyro'

    def connectArduino(self):
        self.arduino = serial.Serial(self.port, self.baudrate, timeout=1)
    
    def disconnectArduino(self):
        self.arduino.close()
    
    def getDistance_test(self):
        self.arduino.write(self.IR_command.encode())
        rdistance = self.arduino.readline().decode().strip()
        if rdistance == '':
            print(rdistance)
            rdistance = self.arduino.readline().decode().strip()
            if rdistance == '':
                print("Distancia invalida")
                return 0
        if rdistance != '':
            rdistance = float(rdistance)
            print(rdistance)
            if rdistance > 20.00 and rdistance < 160.00:
                return rdistance
            else:
                print("Distancia invalida")
                return 0
            
    def getDistance(self):
        self.arduino.write(self.IR_command.encode())
        rdistance = self.arduino.readline().decode().strip()
        if rdistance == '':
            rdistance = self.arduino.readline().decode().strip()
            if rdistance == '':
                print("Distancia invalida")
                return 0
        if rdistance != '':
            rdistance = float(rdistance)
            if rdistance > 15.00 and rdistance < 155.00:
                return rdistance
            else:
                print("Distancia invalida")
                return 0
    
    def getDPS(self):
        self.arduino.write(self.IMUg_command.encode())
        rDPS = self.arduino.readline().decode().strip()
        if rDPS == '':
            rDPS = self.arduino.readline().decode().strip()
        dps = rDPS.split(",")
        xdps = float(dps[0])
        ydps = float(dps[1])
        zdps = float(dps[2])
        return xdps,ydps,zdps
        #return rDPS

def dps2AX(dps):
    AX_speed = dps/0.666
    return round(AX_speed)

def AX2dps(AX_speed):
    dps = AX_speed*0.666
    return dps

def dps2secs(dps, dist = 180): # Usign 180 as the defeault distance for the caracterization to be able to capture the velocity at the right moment
    time_secs = dist/dps
    return time_secs

In [40]:
jointHandler.loadEMERGE()

Succeeded to open the port
Succeeded to change the baudrate
[ID:012] ping Succeeded.


In [50]:
jointHandler.unloadEMERGE()

In [67]:
sensorHandler = SensorHandler(250000)
sensorHandler.connectArduino()

In [71]:
sensorHandler.disconnectArduino()

In [13]:
def dps2AX(dps):
    AX_speed = dps/0.666
    return round(AX_speed)

def AX2dps(AX_speed):
    dps = AX_speed*0.666
    return dps

def dps2secs(dps, dist = 180): # Usign 180 as the defeault distance for the caracterization to be able to capture the velocity at the right moment
    time_secs = dist/dps
    return time_secs

In [68]:
sensorHandler.getDistance()

Distancia invalida


0

In [70]:
sensorHandler.getDPS()

(0.82, 1.02, -0.2)

### Pruebas

In [8]:
i = 1
dps2AX(50)

75

In [9]:
jointHandler.exact_rad

0.017453292519943295

In [10]:
jointHandler.getJointPosition(12)

-0.010226538585904166

En esta linea se define la funcion que se va a utilizar para caracterizar la IMU, la velocidad se puede escribir y luego se usa la funcion de moverse sin perfil de velocidad, por lo que se hace con velocidad constante entonces se mueve de -pi/2 a pi/2 en un tiempo t a velocidad constante. Se va a comparar la velocidad leida por la IMU con la leida del servo. 
El probema es que el movimiento se realiza muy rapido por lo que la medicion de velocidad no se hace siempre en el mismo momento, entre el 30 y el 70% del movimiento. Se va a buscar hacerlo por tiempo

In [15]:
i = -i
target_position = i*math.pi/2
print(target_position)
velocity = 70 # dps
target_vel = dps2AX(velocity)
print(target_vel)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
#time.sleep(0.7)
pos = jointHandler.getJointPosition(12)
dist = abs(target_position - pos)
diff = dist
while diff > 2*jointHandler.exact_rad:
    print("Diff:", diff)
    AX_speed = jointHandler.getJointVelocity(12)
    dps = AX2dps(jointHandler.getJointVelocity(12))
    gyro = sensorHandler.getDPS()
    print(f"Target: {velocity}dps - {target_vel}AX, Present: {AX_speed}AX - {gyro[0]}dps")
    act_pos = jointHandler.getJointPosition(12)
    diff = abs(target_position - act_pos)

-1.5707963267948966
105
Diff: 3.1405699997312033
Target: 70dps - 105AX, Present: 0AX - -68.46dps
Diff: 2.0207640245746847
Target: 70dps - 105AX, Present: -104AX - -66.16dps
Diff: 0.8702784336604539
Target: 70dps - 105AX, Present: -104AX - 2.5dps


Necesito definir un lapso para medir la velocidad, deberia hacerlo en medio movimiento

Para un movimiento de 180 grados se debe calcular el tiempo que toma el motor en realizarlo dependiendo de la velocidad que se defina en dps

Se calcula la velocidad en revoluciones por segundo

In [7]:
def dps2secs(dps, dist = 180): # Usign 180 as the defeault distance for the caracterization to be able to capture the velocity at the right moment
    time_secs = dist/dps
    return time_secs

In [28]:
dps2secs(70)

2.5714285714285716

Se define nuevamente la funcion para realizar las mediciones

In [30]:
i = 1

Baudrate 250 000 // i2C Arduino 400 000

In [ ]:
#sensorHandler = SensorHandler(250000)
#sensorHandler.connectArduino()

In [17]:
sensorHandler.getDPS()

(0.56, 0.83, -0.21)

In [44]:
i = -i
target_position = i*math.pi/2
pres_pos = jointHandler.getJointPosition(12)
velocity = 100 # dps
target_vel = dps2AX(velocity)
print(velocity)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
mov_time = dps2secs(velocity)
capt_time = 0.3 * mov_time # To capture the velocity after 30% of the movement
time.sleep(capt_time)
gyro = sensorHandler.getDPS()
AX_speed = jointHandler.getJointVelocity(12)
dps = AX2dps(AX_speed)
pos = jointHandler.getJointPosition(12)
print(f"Target vel: {velocity}dps = {target_vel}AX") 
print(f"Present vel: {AX_speed}AX = {gyro[2]}dps")
print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")


100
Target vel: 100dps = 150AX
Present vel: -152AX = -97.56dps
Target: -90.0, captured @ -50.09765625000001 deg, started @ 89.94140625000004


Baudrate 100 000
//
i2C Arduino 100 000

In [ ]:
#sensorHandler = SensorHandler(100000)
#sensorHandler.connectArduino()

In [337]:
i = -i
target_position = i*math.pi/2
pres_pos = jointHandler.getJointPosition(12)
velocity = 250 # dps
target_vel = dps2AX(velocity)
print(velocity)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
mov_time = dps2secs(velocity)
capt_time = 0.1 * mov_time # To capture the velocity after 30% of the movement
#time.sleep(capt_time)
gyro = sensorHandler.getDPS()
AX_speed = jointHandler.getJointVelocity(12)
dps = AX2dps(AX_speed)
pos = jointHandler.getJointPosition(12)
print(f"Target vel: {velocity}dps = {target_vel}AX") 
print(f"Present vel: {AX_speed}AX = {gyro[0]}dps")
print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")

250
Target vel: 250dps = 375AX
Present vel: 136AX = 0.34dps
Target: 90.0, captured @ 89.35546874999999 deg, started @ -90.234375


Baudrate 100 000
//
i2C Arduino 400 000

In [83]:
#sensorHandler = SensorHandler(100000)
#sensorHandler.connectArduino()

In [84]:
i = -i
target_position = i*math.pi/2
pres_pos = jointHandler.getJointPosition(12)
velocity = 250 # dps
target_vel = dps2AX(velocity)
print(velocity)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
mov_time = dps2secs(velocity)
capt_time = 0.1 * mov_time # To capture the velocity after 30% of the movement
#time.sleep(capt_time)
gyro = sensorHandler.getDPS()
AX_speed = jointHandler.getJointVelocity(12)
dps = AX2dps(AX_speed)
pos = jointHandler.getJointPosition(12)
print(f"Target vel: {velocity}dps = {target_vel}AX") 
print(f"Present vel: {AX_speed}AX = {gyro[0]}dps")
print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")

250
Target vel: 250dps = 375AX
Present vel: 8AX = -6.37dps
Target: 90.0, captured @ 89.35546874999999 deg, started @ -90.234375


Parece que no es cuestion de la velocidad de comunicacion sino mas del procesamiento de datos del Ardunio (?) De igual forma se utilizara todo a la mayor velocidad serial a 100 000 e i2c a 400 000

In [88]:
i = -i
target_position = i*math.pi/2
pres_pos = jointHandler.getJointPosition(12)
velocity = 250 # dps
target_vel = dps2AX(velocity)
print(velocity)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
mov_time = dps2secs(velocity)
capt_time = 0.1 * mov_time # To capture the velocity after 30% of the movement
#time.sleep(capt_time)
gyro = sensorHandler.getDPS()
AX_speed = jointHandler.getJointVelocity(12)
dps = AX2dps(AX_speed)
pos = jointHandler.getJointPosition(12)
print(f"Target vel: {velocity}dps = {target_vel}AX") 
print(f"Present vel: {AX_speed}AX = {gyro[0]}dps")
print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")

250
Target vel: 250dps = 375AX
Present vel: -8AX = -0.82dps
Target: -90.0, captured @ -90.234375 deg, started @ 89.35546874999999


Se va a probar con el maximo rango de movimiento permitido por el motor (300 grados)

Se cambian las posiciones máximas nuevamente a 0 y 300 grados del motor (0-1023), en terminos del modelo seria 150 a -150, pero por el conector que se usa para la IMU solo se puede de -110 a 110

In [16]:
print(jointHandler.joint_min_position, jointHandler.joint_max_position)
jointHandler.joint_min_position = 0
jointHandler.joint_max_position = 1023
jointHandler.setJointMaxTargetPosition(12)
jointHandler.setJointMinTargetPosition(12)

0 1023


In [342]:
deg_targ = 10
target_position = deg_targ*math.pi/180
jointHandler.setJointAngularPosition(12,-math.pi/2)

In [42]:
jointHandler.loadEMERGE()

Succeeded to open the port
Succeeded to change the baudrate
[ID:012] ping Succeeded.


In [50]:
sensorHandler = SensorHandler(250000)
sensorHandler.connectArduino()

In [17]:
i = 1

gyro a 250 dps

In [52]:
i = -i
deg_targ = 100
target_position = i*deg_targ*math.pi/180
pres_pos = jointHandler.getJointPosition(12)
velocity = 50 # dps
target_vel = dps2AX(velocity)
print(velocity)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
mov_time = dps2secs(velocity, 200)
capt_time = 0.3 * mov_time # To capture the velocity after 30% of the movement
time.sleep(capt_time)
gyro = sensorHandler.getDPS()
AX_speed = jointHandler.getJointVelocity(12)
AX_dps = AX2dps(AX_speed)
pos = jointHandler.getJointPosition(12)
print(f"Target vel: {velocity}dps = {target_vel}AX")
print(f"Present vel: {AX_speed}AX = {AX_dps}AX dps = {gyro[0]}dps")
print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")

50
Target vel: 50dps = 75AX
Present vel: 76AX = 50.616AX dps = 44.98dps
Target: 100.0, captured @ 0.8789062500000161 deg, started @ -100.48828125


In [38]:
sensorHandler.disconnectArduino()

Se habilito un filtro pasabajas de la libreria

In [54]:
sensorHandler = SensorHandler(250000)
sensorHandler.connectArduino()
i = 1

In [57]:
i = -i
deg_targ = 100
target_position = i*deg_targ*math.pi/180
pres_pos = jointHandler.getJointPosition(12)
velocity = 250 # dps
target_vel = dps2AX(velocity)
print(velocity)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
mov_time = dps2secs(velocity, 200)
capt_time = 0.3 * mov_time # To capture the velocity after 30% of the movement
#time.sleep(capt_time)
gyro = sensorHandler.getDPS()
AX_speed = jointHandler.getJointVelocity(12)
AX_dps = AX2dps(AX_speed)
pos = jointHandler.getJointPosition(12)
print(f"Target vel: {velocity}dps = {target_vel}AX")
print(f"Present vel: {AX_speed}AX = {AX_dps}AX dps = {gyro[0]}dps")
print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")

250
Target vel: 250dps = 375AX
Present vel: -96AX = -63.93600000000001AX dps = 44.08dps
Target: -100.0, captured @ -100.48828125 deg, started @ 99.90234375000004


In [ ]:
"""
void loop()
{ 
  if (myICM.dataReady()) 
  {
    myICM.getAGMT();                   // The values are only updated when you call 'getAGMT'
  }            
  if (SERIAL_PORT.available() > 0) 
  {
    String comando = SERIAL_PORT.readString();
    if (comando == 'Gyro') 
    {
      printScaledGyro(&myICM);      // This function takes into account the scale settings from when the measurement was made to calculate the values with units
    }
  }
}
"""

In [93]:
AX2dps(256)

170.496

### Recoleccion de datos del gyro

In [97]:
jointHandler.loadEMERGE()

Succeeded to open the port
Succeeded to change the baudrate
[ID:012] ping Succeeded.


In [169]:
sensorHandler = SensorHandler(250000)
sensorHandler.connectArduino()
i = 1

In [170]:
unfiltered = []
unfiltered_err = []

In [152]:
i = -i
deg_targ = 90
target_position = i*deg_targ*math.pi/180
pres_pos = jointHandler.getJointPosition(12)
velocity = 175 # dps
target_vel = dps2AX(velocity)
print(velocity)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
mov_time = dps2secs(velocity)
capt_time = 0.1 * mov_time # To capture the velocity after 30% of the movement
#time.sleep(capt_time)
gyro = sensorHandler.getDPS()
AX_speed = jointHandler.getJointVelocity(12)
AX_dps = AX2dps(AX_speed)
pos = jointHandler.getJointPosition(12)
print(f"Target vel: {velocity}dps = {target_vel}AX")
print(f"Present vel: {AX_speed}AX = {AX_dps}AX dps = {gyro[0]}dps")
print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")
unfiltered.append((AX_dps,gyro[0]))
unfiltered_err.append(abs(AX_dps-gyro[0]))

175
Target vel: 175dps = 263AX
Present vel: -260AX = -173.16AX dps = -123.18dps
Target: -90.0, captured @ -68.84765625 deg, started @ 89.6484375


In [ ]:
unfiltered_err

In [171]:
for k in range(30):
    i = -i
    deg_targ = 90
    target_position = i*deg_targ*math.pi/180
    pres_pos = jointHandler.getJointPosition(12)
    velocity = 175 # dps
    target_vel = dps2AX(velocity)
    #print(velocity)
    jointHandler.setJointTargetVelocity(12,target_vel)
    jointHandler.setJointAngularPosition(12,target_position)
    mov_time = dps2secs(velocity)
    capt_time = 0.1 * mov_time # To capture the velocity after 30% of the movement
    #time.sleep(capt_time)
    gyro = sensorHandler.getDPS()
    AX_speed = jointHandler.getJointVelocity(12)
    AX_dps = AX2dps(AX_speed)
    pos = jointHandler.getJointPosition(12)
    #print(f"Target vel: {velocity}dps = {target_vel}AX")
    #print(f"Present vel: {AX_speed}AX = {AX_dps}AX dps = {gyro[0]}dps")
    #print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")
    unfiltered.append((AX_dps,gyro[0]))
    unfiltered_err.append(abs(AX_dps-gyro[0]))
    time.sleep(0.5)
un_mean = 0
for err in unfiltered_err:
    un_mean = un_mean + err
un_mean = un_mean/len(unfiltered_err)
print(un_mean)

17.82713333333334


Con filtro

In [166]:
filtered = []
filtered_err = []

In [163]:
i = -i
deg_targ = 90
target_position = i*deg_targ*math.pi/180
pres_pos = jointHandler.getJointPosition(12)
velocity = 175 # dps
target_vel = dps2AX(velocity)
print(velocity)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
mov_time = dps2secs(velocity)
capt_time = 0.1 * mov_time # To capture the velocity after 30% of the movement
#time.sleep(capt_time)
gyro = sensorHandler.getDPS()
AX_speed = jointHandler.getJointVelocity(12)
AX_dps = AX2dps(AX_speed)
pos = jointHandler.getJointPosition(12)
print(f"Target vel: {velocity}dps = {target_vel}AX")
print(f"Present vel: {AX_speed}AX = {AX_dps}AX dps = {gyro[0]}dps")
print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")
filtered.append((AX_dps,gyro[0]))
filtered_err.append(abs(AX_dps-gyro[0]))

175
Target vel: 175dps = 263AX
Present vel: 260AX = 173.16AX dps = 165.98dps
Target: 90.0, captured @ 69.43359375 deg, started @ -90.234375


In [164]:
filtered_err

[17.76400000000001,
 13.147999999999996,
 8.644000000000005,
 0.2540000000000191,
 7.180000000000007]

In [167]:
for k in range(30):
    i = -i
    deg_targ = 90
    target_position = i*deg_targ*math.pi/180
    pres_pos = jointHandler.getJointPosition(12)
    velocity = 175 # dps
    target_vel = dps2AX(velocity)
    #print(velocity)
    jointHandler.setJointTargetVelocity(12,target_vel)
    jointHandler.setJointAngularPosition(12,target_position)
    mov_time = dps2secs(velocity)
    capt_time = 0.1 * mov_time # To capture the velocity after 30% of the movement
    #time.sleep(capt_time)
    gyro = sensorHandler.getDPS()
    AX_speed = jointHandler.getJointVelocity(12)
    AX_dps = AX2dps(AX_speed)
    pos = jointHandler.getJointPosition(12)
    #print(f"Target vel: {velocity}dps = {target_vel}AX")
    #print(f"Present vel: {AX_speed}AX = {AX_dps}AX dps = {gyro[0]}dps")
    #print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")
    filtered.append((AX_dps,gyro[0]))
    filtered_err.append(abs(AX_dps-gyro[0]))
    time.sleep(0.5)
mean = 0
for err in filtered_err:
    mean = mean + err
mean = mean/len(filtered_err)
print(mean) 

9.697600000000003


### Determinar el tiempo para la medicion

In [8]:
i=1
velocity = 185
jointHandler.setJointAngularPosition(12,math.pi/2)
vels = []
dists = []

In [248]:
if velocity >= 15:
    i = -i
    deg_targ = 90
    target_position = i*deg_targ*math.pi/180
    pres_pos = jointHandler.getJointPosition(12)
    #velocity = 115 # dps
    velocity = velocity-10
    target_vel = dps2AX(velocity)
    #print(velocity)
    jointHandler.setJointTargetVelocity(12,target_vel)
    jointHandler.setJointAngularPosition(12,target_position)
    mov_time = dps2secs(velocity)
    #capt_time = 0.1 * mov_time # To capture the velocity after 30% of the movement
    #time.sleep(capt_time)
    gyro = sensorHandler.getDPS()
    AX_speed = jointHandler.getJointVelocity(12)
    AX_dps = AX2dps(AX_speed)
    pos = jointHandler.getJointPosition(12)
    print(f"Target vel: {velocity}dps = {target_vel}AX")
    print(f"Present vel: {AX_speed}AX = {AX_dps}AX dps = {gyro[0]}dps")
    #print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")
    remain = abs(target_position-pos)*180/math.pi
    print(f"@ {velocity} with {remain} deg remaining")
    vels.append(velocity)
    dists.append(remain)
else: 
    print("Velocidad minima")

Target vel: 75dps = 113AX
Present vel: -116AX = -77.256AX dps = -67.76dps
@ 75 with 111.6796875 deg remaining


Para velocidades mayores a 95dps se debe realizar la captura sin ningun tiempo de espera

In [255]:
i=1
velocity = 95
jointHandler.setJointAngularPosition(12,math.pi/2)
vels = []
dists = []

In [274]:
i = -i
deg_targ = 90
target_position = i*deg_targ*math.pi/180
pres_pos = jointHandler.getJointPosition(12)
velocity = 15 # dps
target_vel = dps2AX(velocity)
#print(velocity)
jointHandler.setJointTargetVelocity(12,target_vel)
jointHandler.setJointAngularPosition(12,target_position)
mov_time = dps2secs(velocity)
capt_time = 0.4 * mov_time # To capture the velocity after 30% of the movement
time.sleep(capt_time)
gyro = sensorHandler.getDPS()
AX_speed = jointHandler.getJointVelocity(12)
AX_dps = AX2dps(AX_speed)
pos = jointHandler.getJointPosition(12)
print(f"Target vel: {velocity}dps = {target_vel}AX")
print(f"Present vel: {AX_speed}AX = {AX_dps}AX dps = {gyro[0]}dps")
#print(f"Target: {target_position*180/math.pi}, captured @ {pos*180/math.pi} deg, started @ {pres_pos*180/math.pi}")
remain = abs(target_position-pos)*180/math.pi
print(f"@ {velocity} with {remain} deg remaining")

Target vel: 15dps = 23AX
Present vel: -24AX = -15.984000000000002AX dps = -15.66dps
@ 15 with 98.20312500000003 deg remaining


In [284]:
gyro_velocities = []
AX_velocities = []
input_velocities = []
jointHandler.setJointAngularPosition(12,-math.pi/2)

##### Velocidad de 175 a 95

In [285]:
i = 1
velocity = 175
while velocity >= 95:
    print("Running @: ",velocity)
    target_vel = dps2AX(velocity)
    mov_time = dps2secs(velocity)
    #capt_time = 0.1 * mov_time # To capture the velocity after 30% of the movement
    for k in range(30):
        deg_targ = 90
        target_position = i*deg_targ*math.pi/180
        jointHandler.setJointTargetVelocity(12,target_vel)
        jointHandler.setJointAngularPosition(12,target_position)   
        #time.sleep(capt_time)
        gyro = sensorHandler.getDPS()
        AX_speed = jointHandler.getJointVelocity(12)
        AX_dps = AX2dps(AX_speed)
        # Store in lists
        AX_velocities.append(abs(AX_dps))
        input_velocities.append(abs(velocity))
        gyro_velocities.append(abs(gyro[0]))
        i = -i
        time.sleep(0.5*mov_time) # Waits half of the movement time before starting again
    velocity = velocity - 10
    time.sleep(5)
    print(input_velocities[-1],len(input_velocities))


Running @:  175
175 30
Running @:  165
165 60
Running @:  155
155 90
Running @:  145
145 120
Running @:  135
135 150
Running @:  125
125 180
Running @:  115
115 210
Running @:  105
105 240
Running @:  95
95 270


In [292]:
temp_gyro_velocities = gyro_velocities[:270]
temp_AX_velocities = AX_velocities[:270]
temp_input_velocities = input_velocities[:270]

270

##### Resto de los datos

In [316]:
jointHandler.setJointAngularPosition(12,-math.pi/2)

In [317]:
print(input_velocities[-1],len(input_velocities))

35 480


In [324]:
print(input_velocities[-1],len(input_velocities))
i = 1
velocity = 15
print("Running @: ",velocity)
target_vel = dps2AX(velocity)
mov_time = dps2secs(velocity)
capt_time = 0.5 * mov_time # To capture the velocity after 30% of the movement
for k in range(30):
    deg_targ = 90
    target_position = i*deg_targ*math.pi/180
    jointHandler.setJointTargetVelocity(12,target_vel)
    jointHandler.setJointAngularPosition(12,target_position)   
    time.sleep(capt_time)
    gyro = sensorHandler.getDPS()
    AX_speed = jointHandler.getJointVelocity(12)
    AX_dps = AX2dps(AX_speed)
    # Store in lists
    AX_velocities.append(abs(AX_dps))
    input_velocities.append(abs(velocity))
    gyro_velocities.append(abs(gyro[0]))
    i = -i
    time.sleep(0.8*mov_time) # Waits half of the movement time before starting again
print(input_velocities[-1],len(input_velocities))

25 510
Running @:  15
15 540


In [313]:
AX_velocities = AX_velocities[:480]
input_velocities = input_velocities[:480]
gyro_velocities = gyro_velocities[:480]

### Usar para los demas ejes

In [323]:
vels01 = [85,75]
vels02 = [65,55]
vels04 = [35,25]

i = 1
velocity = 175
while velocity>=15:
    if velocity >= 95:
        time_perc = 0
    elif velocity == 45:
        time_perc = 0.3
    elif velocity == 15:
        time_perc = 0.5
    elif velocity in vels01:
        time_perc = 0.1
    elif velocity in vels02:
        time_perc = 0.2
    elif velocity in vels04:
        time_perc = 0.4
    else:
        time_perc = 0
    
    print("Running @ ",velocity, "dps, time perc: ", time_perc)
    velocity = velocity -10

Running @  175 dps, time perc:  0
Running @  165 dps, time perc:  0
Running @  155 dps, time perc:  0
Running @  145 dps, time perc:  0
Running @  135 dps, time perc:  0
Running @  125 dps, time perc:  0
Running @  115 dps, time perc:  0
Running @  105 dps, time perc:  0
Running @  95 dps, time perc:  0
Running @  85 dps, time perc:  0.1
Running @  75 dps, time perc:  0.1
Running @  65 dps, time perc:  0.2
Running @  55 dps, time perc:  0.2
Running @  45 dps, time perc:  0.3
Running @  35 dps, time perc:  0.4
Running @  25 dps, time perc:  0.4
Running @  15 dps, time perc:  0.5


In [45]:
gyro_velocities = []
AX_velocities = []
input_velocities = []
jointHandler.setJointAngularPosition(12,-math.pi/2)

In [47]:
vels01 = [85,75]
vels02 = [65,55]
vels04 = [35,25]

i = 1
velocity = 175
while velocity>=15:
	if velocity >= 95:
		time_perc = 0
	elif velocity == 45:
		time_perc = 0.3
	elif velocity == 15:
		time_perc = 0.5
	elif velocity in vels01:
		time_perc = 0.1
	elif velocity in vels02:
		time_perc = 0.2
	elif velocity in vels04:
		time_perc = 0.4
	else:
		time_perc = 0
	print("Running @ ",velocity, "dps, time perc: ", time_perc)
	target_vel = dps2AX(velocity)
	mov_time = dps2secs(velocity)
	capt_time = time_perc * mov_time # To capture the velocity after 30% of the movement
	for k in range(30):
		deg_targ = 90
		target_position = i*deg_targ*math.pi/180
		jointHandler.setJointTargetVelocity(12,target_vel)
		jointHandler.setJointAngularPosition(12,target_position)   
		#time.sleep(capt_time)
		gyro = sensorHandler.getDPS()
		AX_speed = jointHandler.getJointVelocity(12)
		AX_dps = AX2dps(AX_speed)
		# Store in lists
		AX_velocities.append(abs(AX_dps))
		input_velocities.append(abs(velocity))
		gyro_velocities.append(abs(gyro[2]))
		i = -i
		time.sleep(mov_time) # Waits half of the movement time before starting again
	velocity = velocity - 10
	time.sleep(5)
	print(input_velocities[-1],len(input_velocities))

Running @  175 dps, time perc:  0
175 30
Running @  165 dps, time perc:  0
165 60
Running @  155 dps, time perc:  0
155 90
Running @  145 dps, time perc:  0
145 120
Running @  135 dps, time perc:  0
135 150
Running @  125 dps, time perc:  0
125 180
Running @  115 dps, time perc:  0
115 210
Running @  105 dps, time perc:  0
105 240
Running @  95 dps, time perc:  0
95 270
Running @  85 dps, time perc:  0.1
85 300
Running @  75 dps, time perc:  0.1
75 330
Running @  65 dps, time perc:  0.2
65 360
Running @  55 dps, time perc:  0.2
55 390
Running @  45 dps, time perc:  0.3
45 420
Running @  35 dps, time perc:  0.4
35 450
Running @  25 dps, time perc:  0.4
25 480
Running @  15 dps, time perc:  0.5
15 510


### Exportar datos

In [35]:
temp_gyro_velocities = gyro_velocities[306:]
temp_AX_velocities = AX_velocities[305:]
temp_input_velocities = input_velocities[305:]

In [48]:
len(gyro_velocities)

510

In [49]:
import csv
# Combina las dos listas en una lista de tuplas
datos_combinados = list(zip(input_velocities, AX_velocities, gyro_velocities, ))

# Especifica el nombre del archivo CSV
nombre_archivo = 'caractIMU_gyroz.csv'

# Abre el archivo CSV en modo de escritura
with open(nombre_archivo, 'w', newline='') as archivo_csv:
    # Crea un objeto escritor CSV
    escritor_csv = csv.writer(archivo_csv)

    # Escribe los encabezados si es necesario
    escritor_csv.writerow(['Velocidad de entrada', 'Velocidad medida por el motor','Velocidad giroscopio'])

    # Escribe los datos
    escritor_csv.writerows(datos_combinados)

print(f'Datos exportados a {nombre_archivo}')

Datos exportados a caractIMU_gyroz.csv
